# Feature Extraction :

## I - Using Weak Labels :

In [181]:
# defining lambda function for reading text
read = lambda src : open(src,"r").read()

# test-reading a paper
text = read("/content/drive/MyDrive/UH - Final Year Project/Dataset/text/0001001v1.txt")
print(text[3000:4000])

d to wave scattering (Chou, Lucas & Stone (1995), Gou, Messiter & Schultz
(1993)), attenuation (Chou & Nelson (1994), Lee et al. (1993)), and Bragg reﬂections
(Chou (1998), Naciri & Mei (1988)). Nonetheless, water wave propagation over random

 
 
 
 
 
 
2

Bal & Chou

static underlying currents that vary on both large and small length scales, and their
interactions, have received relatively less attention.

In this paper, we will only consider static irrotational currents, but derive the transport
equations for surface waves in the presence of underlying ﬂows that vary on both long
and short (on the order of the wavelength) length scales. Rather than computing wave
scattering from speciﬁc static ﬂow conﬁgurations (Gerber (1993), Trulson & Mei (1993),
Fabrikant & Raevsky (1994)), we take a statistical approach by considering ensemble
averages over realisations of the static randomness. Diﬀerent statistical approaches have
been applied to wave propagation over a random depth (Elter & 

In [182]:
import re
# getting functions using regex
func_name = r"α-ωA-Za-zΑ-Ω0-9ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
func_var = r"A-Za-zα-ωΑ-ΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"

func_pattern = fr"(.|)([{func_name}]{{1,3}}(′|.|)\([{func_var}](,\s*[{func_var}])*\))"

matches = re.findall(func_pattern, text)
for match in matches:
    print(match[1])

W(x, k, t)
W(x, k, t)
U1(x, z)
U2(x, z)
Uz(x, z)
V(x, z)
U(x, z)
Uz(x, z)
V(x, z)
η(x)
v(x, z)
η(x, t)
V(x, z)
Φ(x, z)
v(x, z, t)
Φ(x, z)
Φ(x, z)
O(L)
O(L)
O(ε)
η(x)
η(x, t)
ζ(x, t)
tη(x, t)
U(x, ζ)
ζ(x, t)
z(x, ζ)
∂zϕ(x, ζ, t)
η(x, t)
η(x)
η(x)
tη(x, t)
η(x, t)
ρ∂tϕ(x, z, t)
ρU(x, z)
zϕ(x, z, t)
xϕ(x, z, t)
η(x, t)
gη(x, t)
η(x)
η(x, t)
O(L)
U(x)
U(x)
O(ε)
U(x)
tη(x, t)
U(x)
η(x)
tϕ(x, t)
U(x)
xϕ(x, t)
xϕ(x, t)
η(x, t)
ere (i, j)
δU(x)
Ui(x)
1η(x, t)
η(x, t)
η(q, t)
U(x)
U(q)
U(q)
ij (x)
ij (q)
η(x, t)
tη(k, t)
U(q)
δU(q)
tϕ(k, t)
U(q)
U(q)
δU(q)
Ui(p)
Uj(q)
η(k)
Rij (x)
U(x)
ω(k)
U(x)
η(x)
ij (x, k, t)
ij (p, k, t)
η(x)
η(x)
U(x, z)
zUz(x, z)
v(x, z)
U(x, z)
zUz(x, z)
η(k)
η(x, t)
A(k)
W(k, t)
A11(k)
A22(k)
W(x, k, t)
W(p, k, t)
ψj(k, t)
iLjℓ(k)
ℓ(k, t)
U(q)
L(k)
L(k)
δU(q)
Wij (p, k, t)
iWiℓ(p, k, t)
ℓj (p, k, t)
U(q)
U(q)
U(q)
U(q)
W(x, k, t)
0(p, ξ, k, t)
2(p, ξ, k, t)
εW1(p, ξ, k, t)
W(p, k, t)
1L0(k)
L1(k, p)
O(ε)
0(k)
L1(k, p)
f (k)
f (k)
0(p, ξ, k, t)
W0(p, ξ, k, t)
α(k)
α(k)


## II - Using custom Tokenizer :

In [183]:
import re
text1 = "This is a text with f(x) and g(x, y) and aǫ(x, k, t) functions."
text = "we deﬁne the ﬂow at the surface by U(x), z = 0..."
# Find math functions
matches = re.findall(func_pattern, text)
# getting only second-group matches (the functions)
functions = [match[1] for match in matches]
func_saver = iter(functions.copy())
# Replace math functions with temporary markers
for func in functions:
  text = text.replace(func, "[FUNC]")

# Tokenize the rest of the text (you can use your preferred tokenizer)
tokens = text.split()
print(f"tokens before : {tokens}")

# Replace temporary markers with original functions
pattern = re.compile(r"(\[FUNC\]).")

#new_tokens = [next(func_saver) if pattern.match(token) else token for token in tokens]
new_tokens = [next(func_saver) if '[FUNC]' in token else token for token in tokens]
print(f"tokens after : {new_tokens}")

tokens before : ['we', 'deﬁne', 'the', 'ﬂow', 'at', 'the', 'surface', 'by', '[FUNC],', 'z', '=', '0...']
tokens after : ['we', 'deﬁne', 'the', 'ﬂow', 'at', 'the', 'surface', 'by', 'U(x)', 'z', '=', '0...']


In [184]:
from spacy.tokenizer import Tokenizer


def my_tokenizer(text) :

  # defining function pattern
  func_name = r"α-ωA-Za-zΑ-Ω0-9ℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
  func_var = r"A-Za-zα-ωΑ-ΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΣΦΨΩÅℏ∞∘∂∫∮∯∇αβγ∅"
  func_pattern = fr"(.|)([{func_name}]{{1,3}}(′|.|)\([{func_var}](,\s*[{func_var}])*\))"

  # getting math functions
  matches = re.findall(func_pattern, text)

  # getting only second-group matches (the functions)
  functions = [match[1] for match in matches]
  func_saver = iter(functions.copy())

  # Replace math functions with temporary markers
  for func in functions:
    text = text.replace(func, "[FUNC]")

  # Tokenize the rest of the text
  tokens = text.split()

  # Replace temporary markers with original functions
  new_tokens = [next(func_saver) if '[FUNC]' in token else token for token in tokens]

  return new_tokens

In [185]:
def create_spacy_tokenizer(nlp):
    def custom_tokenizer(text):
        tokens = my_tokenizer(text)
        return Doc(nlp.vocab, words=tokens)
    return custom_tokenizer

In [186]:
import spacy
from spacy.tokens import Doc

nlp = spacy.load("en_core_web_sm")

# adding custom tokenizer
nlp.tokenizer = create_spacy_tokenizer(nlp)

text = "This is a text with f(x) and g(x, y) and aǫ(x, k, t) functions."

# Process the text
doc = nlp(text)

# Iterate over the tokens in the processed doc
for token in doc:
    print(token.text)

This
is
a
text
with
f(x)
and
g(x, y)
and
aǫ(x, k, t)
functions.


## II - Using custom NER using Regex weak labels :

In [187]:
# testing basic NER
import spacy

#sample text
text = "The value of epsilon is known as ε = 32 "

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

for ent in doc.ents :
  print(ent.text,ent.label_)

32 CARDINAL


In [272]:
# Define symbols to be tagged with custom NER tags
mathematical_symbols = [
    'α', 'β', 'γ', 'δ', 'ε', 'ζ', 'η', 'θ', 'ι', 'κ', 'λ', 'μ', 'ν', 'ξ', 'ο', 'π', 'ρ', 'σ', 'τ', 'υ', 'φ', 'χ', 'ψ', 'ω',
    'Α', 'Β', 'Γ', 'Δ', 'Ε', 'Ζ', 'Η', 'Θ', 'Ι', 'Κ', 'Λ', 'Μ', 'Ν', 'Ξ', 'Ο', 'Π', 'Ρ', 'Σ', 'Τ', 'Υ', 'Φ', 'Χ', 'Ψ', 'Ω',
    'ℰ', 'ℓ', 'ℒ', 'ℳ', 'ø', 'ℂ', 'ℕ', 'ℙ', 'ℚ', 'ℝ', 'ℤ','Γ', 'Δ', 'Λ', 'Ξ', 'Π', 'Φ', 'Ψ', 'Ω','Å', 'ℏ', '∞',
    '∂', '∮', '∯', '∇','∅','˜','µ','ǫ','ℋ', 'ℨ', 'ℛ']

math_var = ''.join(mathematical_symbols)
mathematical_operations = ['∫', '∑', '∏', '√', '+', '-', '*', '/', '=', '^', '%','∩', '∪', '⊂', '⊆', '∈', '∉','∘' , "≡","<",">","↔","|"]

mathematical_verbs = ["define","called", "defined as", "used for", "termed as", "represents", "denotes", "stands for", "refers to",
                      "corresponds to", "signifies", "designates", "characterizes as","identified as", "labelled as", "named","outlined as",
                      "identified with", "associated with", "portrays", "symbolizes", "denoted", "denotes","given by", "known as","deﬁne","we choose","taken to be"]

math_sym = fr"\b([A-Za-zα-ωΑ-Ω|{math_var}]{{1}}(\d)*)\b"

# math structures (functions,sets,lists,vectors)
#\([{(A-Za-zα-ωΑ-Ω|αβγδεζηθικλμνξοπρστυφχψωΑΒΓΔΕΖΗΘΙΚΛΜΝΞΟΠΡΣΤΥΦΧΨΩℰℓℒℳøℂℕℙℚℝℤΓΔΛΞΠΦΨΩÅℏ∞∂∮∯∇∅˜µǫℋℨℛ/s*,|\d{1}/s*,)*}]*\)
sets = r"\{[{(A-Za-zα-ωΑ-Ω|{math_var}{{1}}/s*,|\d{{1}}/s*,)*}]*\}"
lists = r"\[[{(A-Za-zα-ωΑ-Ω|{math_var}{{1}}/s*,|\d{{1}}/s*,)*}]*\]"
vectors = r"\([{(A-Za-zα-ωΑ-Ω|{math_var}{{1}}/s*,|\d{{1}}/s*,)*}]*\)"



regex_patterns = [(func_pattern,"MATH_VAR"), (sets,"MATH_VAR"),(lists,"MATH_VAR") , (vectors,"MATH_VAR"),(math_sym,"MATH_VAR")]

In [189]:
import spacy
from spacy.tokens import Span
import pandas as pd

# defining custom NER component
@spacy.Language.component("parsing_math_entities")

def parsing_math_entities(doc):
    entities = list(doc.ents)
    for token in doc:
        if token.text in mathematical_symbols:
          entities.append(Span(doc, token.i, token.i + 1, label="MATH_VAR"))
        if token.text in mathematical_operations :
          entities.append(Span(doc,token.i,token.i+1,label="MATH_OPP"))
        elif token.text in mathematical_verbs :
          entities.append(Span(doc,token.i,token.i+1,label="MATH_VRB"))


    # regex patterns
    for pattern , label in regex_patterns:
        for match in re.finditer(pattern, doc.text):
            start, end = match.span()

            # getting rid of white-space start in matches
            if doc.text[start] == " " :
              start_token = doc.char_span(start+1, end)

            else :
              start_token = doc.char_span(start, end)

            if start_token:
                entities.append(Span(doc, start_token.start, start_token.end, label=label))

    # adding more entities
    entities = spacy.util.filter_spans(entities)

    # Updating the document's entities
    doc.ents = entities

    return doc

In [190]:
# Example text
text = "This function denoted as f(x) = k(x,y) and z(x, y,w)"
text2 = "we the ﬂow at the surface by U(x, z = 0..."

nlp = spacy.load("en_core_web_sm")

# adding custom tokenizer
nlp.tokenizer = create_spacy_tokenizer(nlp)

# Adding the custom NER component to the pipeline
nlp.add_pipe("parsing_math_entities",before="ner")

# Process the text with the updated pipeline
doc = nlp(text)

entities = [(ent.text, ent.label_) for ent in doc.ents]
df = pd.DataFrame(entities, columns=['Entity', 'Label'])
df

,Entity,Label
0,denoted,MATH_VRB
1,f(x),MATH_VAR
2,=,MATH_OPP
3,"k(x,y)",MATH_VAR
4,"z(x, y,w)",MATH_VAR


In [191]:
# defining lambda function for reading text
read = lambda src : open(src,"r").read()

# test-reading a paper
text = read("/content/drive/MyDrive/UH - Final Year Project/Dataset/cleaned_text/0001001v1.txt")
print(text)

  ] - .
[ / :  Under consideration for publication in J. Fluid Mech.
Capillary-gravity wave transport over spatially random drift  By G U I L L A U M E B A L∗ and T O M C H O U† ∗ Department of Mathematics, University of Chicago, Chicago, IL†Department of Mathematics, Stanford University, Stanford, CA (Received 2 February 2008)  We derive transport equations for the propagation of water wave action in the pres- ence of a static, spatially random surface drift.
Using the Wigner distribution W(x, k, t) to represent the envelope of the wave amplitude at position x contained in waves with wavevector k, we describe surface wave transport over static ﬂows consisting of two length scales; one varying smoothly on the wavelength scale, the other varying on a scale com- parable to the wavelength.
The spatially rapidly varying but weak surface ﬂows augment the characteristic equations with scattering terms that are explicit functions of the cor- relations of the random surface currents.
These sca

In [192]:
# Process the text with the updated pipeline
doc = nlp(text)

entities = [(ent.text, ent.label_) for ent in doc.ents]
df = pd.DataFrame(entities, columns=['Entity', 'Label'])
df

,Entity,Label
0,-,MATH_OPP
1,/,MATH_OPP
2,J. Fluid Mech.,PERSON
3,G,MATH_VAR
4,U,MATH_VAR
...,...,...
2037,∇,MATH_VAR
2038,∇,MATH_VAR
2039,=,MATH_OPP
2040,∇,MATH_VAR


## Visualising Custom NER

In [193]:
from spacy import displacy

displacy.render(doc, style="ent", jupyter=True)

# Feature Extraction :

- [ ] Clean / get rid of empty entities
- [ ] Get rid of "A"s and "I"s that are not variables
- [ ] Get variable-name using `math_vrb`
- [ ] Get variable-name using `math_var`

# Creating the dataframe :

In [194]:
# defining sample text snippet
sample_text = """We eventually take the limit ε  z/ε and t  x/ε, z  →  →  →  →   ≡  ∼  Since Uz(x, z  0)  ≈  ≈  0, we deﬁne the ﬂow at the surface by  U(x, z = 0)  ≡  U(x) + √εδU(x/ε).
(2.5)  In these rescaled coordinates, U(x) denotes surface ﬂows varying on length scales of O much greater than a typical wavelength, while δU(x/ε) varies over lengths of O(ε) comparable to a typical wavelength.
The amplitude of the slowly varying ﬂow U(x) is O(ε0), while that of the rapidly varying ﬂow δU(x/ε), is assumed to be of O(√ε).
A more detailed discussion of the physical motivation for considering the √ε scaling is deferred to the Results and Discussion.
After rescaling, the boundary conditions (2.4) evaluated at z = 0 become  ∂tη(x, t) +  ∇  ·  U(x) + √εδU(x/ε) (cid:1)  (cid:2)(cid:0)  η(x) (cid:3)  = lim  z→0− ∂zϕ(x, 0)  ∂tϕ(x, t) + U(x)  xϕ(x, t) + √εδU(x/ε)  xϕ(x, t) = ε∆η(x, t)  · ∇  · ∇  (2.6) Although drift that varies slowly along one wavelength can be treated with characteristics and WKB theory, random ﬂows varying on the wavelength scale require a statistical approach.
Without loss of generality, we choose δU to have zero mean and an isotropic x′ two-point correlation function ), where (i, j) = (1, 2) and | denotes an ensemble average over realisations of δU(x).
.
.
.
δUi(x)δUj (x′)
We now deﬁne the spatial Fourier decompositions for the dynamical wave variables  Rij( |  i ≡  −  ε−1η(x, t).
−  ϕ(x,  −  h 6 z 6 ζ, t) =  ϕ(q, t)e−iq·x cosh q(h + z)  cosh qh  Zq  ,  η(x, t) =  Zq  η(q, t)e−iq·x,  the static surface ﬂows  U(x) =  Zq  U(q)e−iq·x,  δU  ε (cid:17)  (cid:16)  =  Zq  δU(q)e−iq·x/ε,  and the correlations  Rij (x) =  Zq  Rij (q)e−iq·x,  (2.7)  (2.8)  (2.9)  where q = (q1, q2) is an in-plane two dimensional wavevector, q  2, and (2π)−2 dq1dq2.
The Fourier integrals for η exclude q = 0 due to the incompress- q ≡ x η(x, t) = 0, while the q = 0 mode for ϕ gives an irrelevant constant ibility constraint shift to the velocity potential.
Note that ϕ in (2.7) manifestly satisﬁes (2.1).
Substituting  |  ≡ |  1 + q2 q2   =
(2.8) into the boundary conditions (2.4), we obtain,  Water wave transport  ∂tη(k, t)   −  Zq  η(k  −  q)U(q)   ·  −  i√ε  Zq  η(k  −  q/ε)δU(q)  ·  k = ϕ(k, t) k tanh εkh  ∂tϕ(k, t)   −  Zq  U(q)  (k  ·  −  q) ϕ(k  q)  −  −  i√ε  Zq  δU(q)  (k  ·  −  q/ε) ϕ(k  q/ε)  −  where the δU(q) are correlated according to  δUi(p)δUj(q)  )δ(p + q).
= Rij ( | |  (εk2 + ε−1)η(k).
=  −  (2.10)  (2.11)  Since the correlation Rij (x) is symmetric in i , Rij( | | | In the case where δU = 0 and U(x)  ) is real.
|  −  U0 is strictly uniform, equations (2.10) can be simpliﬁed by assuming a e−iωt dependence for all dynamical variables.
Uniform drift yields the familiar capillary-gravity wave dispersion relation  ≡  j, and depends only upon the magnitude  ↔  ω(k) =  (k3 + k) tanh kh + U0 ·    Ω(k) + U0 ·  k.  ≡  (2.12)  However, for what follows, we wish to derive transport equations for surface waves (action, energy, intensity) in the presence of a spatially varying drift containing two length scales: U = U(x) + √εδU(x/ε).
"""

In [195]:
# testing split fix :
test_sent = "this is a test sentence where f(x,y) , and g(x,y) are being tested"
pattern = r',\s*(?![^()]*\))'
split_list = re.split(pattern, test_sent)
print(split_list)

['this is a test sentence where f(x,y) ', 'and g(x,y) are being tested']


In [196]:
def custom_NER_tokenization(text , NER = False) :
  # Load the spaCy model
  nlp = spacy.load("en_core_web_sm")

  # adding custom tokenizer
  nlp.tokenizer = create_spacy_tokenizer(nlp)

  # Adding the custom NER component to the pipeline
  if NER :
    nlp.add_pipe("parsing_math_entities",before="ner")

  # Process the text
  doc = nlp(text)

  return doc

In [197]:
# creating the dataframe
import nltk
from nltk.tokenize import word_tokenize , sent_tokenize
nltk.download('punkt')

labels = ["MATH_VAR","MATH_STRUCT", "MATH_SYMBOL" , "MATH_OPP" ,"MATH_VRB"]

nlp = spacy.load("en_core_web_sm")

# adding custom tokenizer
nlp.tokenizer = create_spacy_tokenizer(nlp)

# Adding the custom NER component to the pipeline
nlp.add_pipe("parsing_math_entities",before="ner")

sentences = sent_tokenize(sample_text)
# defining variables
sent_of_interest = {"sentences":[],"entities":[] ,"labels" :[]}
pattern = r'[,=]\s*(?![^()]*\))'
for sent in sentences :
  sub_sents = re.split(pattern, sent)
  for sub_sent in sub_sents :
    doc = nlp(sub_sent)
    ent_list = []
    label_list = []
    for ent in doc.ents :
      if ent.label_ in labels :
        ent_list.append(ent.text)
        label_list.append(ent.label_)
    sent_of_interest["sentences"].append(sub_sent)
    sent_of_interest["entities"].append(ent_list)
    sent_of_interest["labels"].append(label_list)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [198]:
df = pd.DataFrame(sent_of_interest)
df.iloc[0].sentences

'We eventually take the limit ε  z/ε and t  x/ε'

In [199]:
df

,sentences,entities,labels
0,We eventually take the limit ε z/ε and t x/ε,"[ε, t]","[MATH_VAR, MATH_VAR]"
1,"z → → → → ≡ ∼ Since Uz(x, z 0) ≈ ≈ 0","[z, ≡, z]","[MATH_VAR, MATH_OPP, MATH_VAR]"
2,"we deﬁne the ﬂow at the surface by U(x, z = 0...","[deﬁne, z, =, ≡, U(x), +]","[MATH_VRB, MATH_VAR, MATH_OPP, MATH_OPP, MATH_..."
3,(2.5) In these rescaled coordinates,[],[]
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O, a]","[MATH_VAR, MATH_VRB, MATH_VAR, MATH_VAR]"
...,...,...,...
57,and depends only upon the magnitude ↔ ω(k),"[↔, ω(k)]","[MATH_OPP, MATH_VAR]"
58,(k3 + k) tanh kh + U0 · Ω(k) + U0 · k. ≡ ...,"[+, +, U0, +, U0, ≡]","[MATH_OPP, MATH_OPP, MATH_VAR, MATH_OPP, MATH_..."
59,for what follows,[],[]
60,we wish to derive transport equations for surf...,"[a, U]","[MATH_VAR, MATH_VAR]"


In [200]:
import nltk
from nltk.tokenize import word_tokenize , sent_tokenize

# downloading punk dict
nltk.download('punkt')

def create_df(text,pattern,labels) :

  # getting sentence tokens
  sentences = sent_tokenize(text)

  #defining dictionary
  sent_of_interest = {"sentences":[],"entities":[] ,"labels" :[]}
  # looping over sentences
  for sent in sentences :
    # getting sub-sentences (split on "," unless it's between ())
    sub_sents = re.split(pattern, sent)
    # looping over sub-sentences
    for sub_sent in sub_sents :
      # getting tokens out of sub-sentences
      doc = custom_NER_tokenization(sub_sent,True)
      # defining entity/label lists
      ent_list = []
      label_list = []
      # looping over token entities
      for ent in doc.ents :
        # taking only labels in the math-label list
        if ent.label_ in labels :
          # adding labels and entities to their lists
          ent_list.append(ent.text)
          label_list.append(ent.label_)
      # adding the lists to dictionary keys
      sent_of_interest["sentences"].append(sub_sent)
      sent_of_interest["entities"].append(ent_list)
      sent_of_interest["labels"].append(label_list)

  # creating the dataframe
  df = pd.DataFrame(sent_of_interest)

  return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [201]:
 # defining variables
labels = ["MATH_VAR","MATH_STRUCT", "MATH_SYMBOL" , "MATH_OPP" ,"MATH_VRB"]
pattern = r'[,=]\s*(?![^()]*\))'

df = create_df(sample_text,pattern,labels)
df

,sentences,entities,labels
0,We eventually take the limit ε z/ε and t x/ε,"[ε, t]","[MATH_VAR, MATH_VAR]"
1,"z → → → → ≡ ∼ Since Uz(x, z 0) ≈ ≈ 0","[z, ≡, z]","[MATH_VAR, MATH_OPP, MATH_VAR]"
2,"we deﬁne the ﬂow at the surface by U(x, z = 0...","[deﬁne, z, =, ≡, U(x), +]","[MATH_VRB, MATH_VAR, MATH_OPP, MATH_OPP, MATH_..."
3,(2.5) In these rescaled coordinates,[],[]
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O, a]","[MATH_VAR, MATH_VRB, MATH_VAR, MATH_VAR]"
...,...,...,...
57,and depends only upon the magnitude ↔ ω(k),"[↔, ω(k)]","[MATH_OPP, MATH_VAR]"
58,(k3 + k) tanh kh + U0 · Ω(k) + U0 · k. ≡ ...,"[+, +, U0, +, U0, ≡]","[MATH_OPP, MATH_OPP, MATH_VAR, MATH_OPP, MATH_..."
59,for what follows,[],[]
60,we wish to derive transport equations for surf...,"[a, U]","[MATH_VAR, MATH_VAR]"


# Adding POS tags to dataframe :

In [202]:
import spacy

sample_text = "The speed v is known as the velocity"
sam_text = "we deﬁne the ﬂow at the surface by U(x), z = 0..."

# getting tokens from custom tokenizer
doc = custom_NER_tokenization(sam_text)

# Print POS tags
for token in doc:
    print(f"Word: {token.text} | POS: {token.pos_} | Tag: {token.tag_} | desc: {spacy.explain(token.tag_)}")

Word: we | POS: PRON | Tag: PRP | desc: pronoun, personal
Word: deﬁne | POS: VERB | Tag: VBP | desc: verb, non-3rd person singular present
Word: the | POS: DET | Tag: DT | desc: determiner
Word: ﬂow | POS: NOUN | Tag: NN | desc: noun, singular or mass
Word: at | POS: ADP | Tag: IN | desc: conjunction, subordinating or preposition
Word: the | POS: DET | Tag: DT | desc: determiner
Word: surface | POS: NOUN | Tag: NN | desc: noun, singular or mass
Word: by | POS: ADP | Tag: IN | desc: conjunction, subordinating or preposition
Word: U(x) | POS: PROPN | Tag: NNP | desc: noun, proper singular
Word: z | POS: PROPN | Tag: NNP | desc: noun, proper singular
Word: = | POS: SYM | Tag: SYM | desc: symbol
Word: 0... | POS: NUM | Tag: CD | desc: cardinal number


In [203]:
def POS_tag (text) :
  doc = custom_NER_tokenization(text)
  pos_tags = {"words": [] , "POS" : [] , "tag" : []}

  for token in doc:
    pos_tags["words"].append(token.text)
    pos_tags["POS"].append(token.pos_)
    pos_tags["tag"].append(token.tag_)

  return pos_tags

In [204]:
# testing POS tag function on sample text
text2 = "The flow of surface v is defined to be"
tags = POS_tag(text2)
tags["words"]

['The', 'flow', 'of', 'surface', 'v', 'is', 'defined', 'to', 'be']

In [205]:
# adding the tags to the dataframe
tag_list = []

for sent in df["sentences"] :
  tags = POS_tag(sent)
  tag_list.append(tags["tag"])

df["tags"] = tag_list
df.head()

,sentences,entities,labels,tags
0,We eventually take the limit ε z/ε and t x/ε,"[ε, t]","[MATH_VAR, MATH_VAR]","[PRP, RB, VBP, DT, NN, NN, NNP, CC, NNP, NNP]"
1,"z → → → → ≡ ∼ Since Uz(x, z 0) ≈ ≈ 0","[z, ≡, z]","[MATH_VAR, MATH_OPP, MATH_VAR]","[NN, SYM, ``, ``, ``, FW, NNS, IN, JJ, NN, NNS..."
2,"we deﬁne the ﬂow at the surface by U(x, z = 0...","[deﬁne, z, =, ≡, U(x), +]","[MATH_VRB, MATH_VAR, MATH_OPP, MATH_OPP, MATH_...","[PRP, VBP, DT, NN, IN, DT, NN, IN, NN, NN, NN,..."
3,(2.5) In these rescaled coordinates,[],[],"[-LRB-, IN, DT, JJ, NNS]"
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O, a]","[MATH_VAR, MATH_VRB, MATH_VAR, MATH_VAR]","[NNP, VBZ, NN, VBZ, VBG, IN, NN, NNS, IN, UH, ..."


In [206]:
def add_tags(df) :
  tag_list = []

  for sent in df["sentences"] :
    tags = POS_tag(sent)
    tag_list.append(tags["tag"])

  df["tags"] = tag_list

## Dataframe Cleaning :

### Getting rid of false variables :

#### 1- Testing POS taging difference :

In [207]:
text_A_1 = "A football player is named footballer"
text_A_2 = "U(x) denotes surface ﬂows varying on length scales of O much greater than a typical wavelength"

tags_A_1 = POS_tag(text_A_1)
tags_A_2 = POS_tag(text_A_2)

print("First Case A :\n ")
print(tags_A_1["words"])
print(tags_A_1["tag"])
print("-"*50)
print("Second Case A :\n ")
print(tags_A_2["words"])
print(tags_A_2["tag"])

First Case A :
 
['A', 'football', 'player', 'is', 'named', 'footballer']
['DT', 'NN', 'NN', 'VBZ', 'VBN', 'NN']
--------------------------------------------------
Second Case A :
 
['U(x)', 'denotes', 'surface', 'ﬂows', 'varying', 'on', 'length', 'scales', 'of', 'O', 'much', 'greater', 'than', 'a', 'typical', 'wavelength']
['NNP', 'VBZ', 'NN', 'VBZ', 'VBG', 'IN', 'NN', 'NNS', 'IN', 'UH', 'RB', 'JJR', 'IN', 'DT', 'JJ', 'NN']


In [208]:
text_I_1 = "I love comedy and i love it so much"
text_I_2 = "the flow of electricity i is equal to "

tags_I_1 = POS_tag(text_I_1)
tags_I_2 = POS_tag(text_I_2)

print("First Case I :\n ")
print(tags_I_1["words"])
print(tags_I_1["tag"])
print("-"*50)
print("Second Case I :\n ")
print(tags_I_2["words"])
print(tags_I_2["tag"])

First Case I :
 
['I', 'love', 'comedy', 'and', 'i', 'love', 'it', 'so', 'much']
['PRP', 'VBP', 'NN', 'CC', 'PRP', 'VBP', 'PRP', 'RB', 'RB']
--------------------------------------------------
Second Case I :
 
['the', 'flow', 'of', 'electricity', 'i', 'is', 'equal', 'to']
['DT', 'NN', 'IN', 'NN', 'PRP', 'VBZ', 'JJ', 'IN']


#### 2- Testing getting rid of target words :

In [209]:
target_words_A = {'A','a'}
target_words_I = {'i' ,'I'}


target_sent = []
for sent , ent , labels , tags in zip(df["sentences"],df["entities"],df["labels"],df["tags"]) :
  doc = custom_NER_tokenization(sent)
  token_list = [token for token in doc]

  for token in doc :
    token_index = token_list.index(token)
    if (token.text in target_words_A) & (tags[token_index] == "DT") :
      ent_index = ent.index(token.text)
      ent.pop(ent_index)
      labels.pop(ent_index)


    if (token.text in target_words_I) & (token_index == 0) :
      print(f"entitites before : {ent}")
      print(f"labels before : {labels}")
      ent_index = ent.index(token.text)
      ent.pop(ent_index)
      labels.pop(ent_index)
      print(f"entitites after : {ent}")
      print(f"labels after : {labels}")

In [210]:
df

,sentences,entities,labels,tags
0,We eventually take the limit ε z/ε and t x/ε,"[ε, t]","[MATH_VAR, MATH_VAR]","[PRP, RB, VBP, DT, NN, NN, NNP, CC, NNP, NNP]"
1,"z → → → → ≡ ∼ Since Uz(x, z 0) ≈ ≈ 0","[z, ≡, z]","[MATH_VAR, MATH_OPP, MATH_VAR]","[NN, SYM, ``, ``, ``, FW, NNS, IN, JJ, NN, NNS..."
2,"we deﬁne the ﬂow at the surface by U(x, z = 0...","[deﬁne, z, =, ≡, U(x), +]","[MATH_VRB, MATH_VAR, MATH_OPP, MATH_OPP, MATH_...","[PRP, VBP, DT, NN, IN, DT, NN, IN, NN, NN, NN,..."
3,(2.5) In these rescaled coordinates,[],[],"[-LRB-, IN, DT, JJ, NNS]"
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O]","[MATH_VAR, MATH_VRB, MATH_VAR]","[NNP, VBZ, NN, VBZ, VBG, IN, NN, NNS, IN, UH, ..."
...,...,...,...,...
57,and depends only upon the magnitude ↔ ω(k),"[↔, ω(k)]","[MATH_OPP, MATH_VAR]","[CC, VBZ, RB, IN, DT, NN, NNP, NNP]"
58,(k3 + k) tanh kh + U0 · Ω(k) + U0 · k. ≡ ...,"[+, +, U0, +, U0, ≡]","[MATH_OPP, MATH_OPP, MATH_VAR, MATH_OPP, MATH_...","[NFP, NFP, NFP, NN, NNP, SYM, NN, NFP, ADD, CC..."
59,for what follows,[],[],"[IN, WP, VBZ]"
60,we wish to derive transport equations for surf...,[U],[MATH_VAR],"[PRP, VBP, TO, VB, NN, NNS, IN, NN, NNS, NNP, ..."


## Getting rid of empty entities :

In [211]:
condition = df['entities'].apply(lambda ent: len(ent) > 0)
df = df[condition]

In [212]:
df

,sentences,entities,labels,tags
0,We eventually take the limit ε z/ε and t x/ε,"[ε, t]","[MATH_VAR, MATH_VAR]","[PRP, RB, VBP, DT, NN, NN, NNP, CC, NNP, NNP]"
1,"z → → → → ≡ ∼ Since Uz(x, z 0) ≈ ≈ 0","[z, ≡, z]","[MATH_VAR, MATH_OPP, MATH_VAR]","[NN, SYM, ``, ``, ``, FW, NNS, IN, JJ, NN, NNS..."
2,"we deﬁne the ﬂow at the surface by U(x, z = 0...","[deﬁne, z, =, ≡, U(x), +]","[MATH_VRB, MATH_VAR, MATH_OPP, MATH_OPP, MATH_...","[PRP, VBP, DT, NN, IN, DT, NN, IN, NN, NN, NN,..."
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O]","[MATH_VAR, MATH_VRB, MATH_VAR]","[NNP, VBZ, NN, VBZ, VBG, IN, NN, NNS, IN, UH, ..."
5,while δU(x/ε) varies over lengths of O(ε) comp...,[O(ε)],[MATH_VAR],"[IN, NN, VBZ, IN, NNS, IN, NN, JJ, IN, DT, JJ,..."
6,The amplitude of the slowly varying ﬂow U(x) i...,[U(x)],[MATH_VAR],"[DT, NN, IN, DT, RB, VBG, PRP$, NNP, VBZ, NNP]"
11,the boundary conditions (2.4) evaluated at z,[z],[MATH_VAR],"[DT, JJ, NNS, -LRB-, VBD, IN, NN]"
12,"0 become ∂tη(x, t) + ∇ · U(x) + √εδU(x/ε) ...","[∂tη(x, t), +, ∇, U(x), +, η(x)]","[MATH_VAR, MATH_OPP, MATH_VAR, MATH_VAR, MATH_...","[NFP, VB, NNP, NNP, NN, NFP, NNP, CC, NNP, -LR..."
13,"lim z→0− ∂zϕ(x, 0) ∂tϕ(x, t) + U(x) xϕ(x, t...","[∂tϕ(x, t), +, U(x), xϕ(x, t), +, xϕ(x, t)]","[MATH_VAR, MATH_OPP, MATH_VAR, MATH_VAR, MATH_...","[NNP, NNP, NNP, VBZ, NNP, CC, NNP, NNP, NFP, N..."
14,"ε∆η(x, t) · ∇ · ∇ (2.6) Although drift that...","[η(x, t), ∇, ∇]","[MATH_VAR, MATH_VAR, MATH_VAR]","[IN, NFP, ADD, NFP, ADD, NFP, IN, NN, WDT, VBZ..."


## Checking If sentence is real :

- This means checking if sentence has real words

In [213]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

my_list = ["−" ,"ϕ(x", "−", "h","6","z" ,"6" ,"ζ", "t",")" ,"=", "ϕ(q, t)","e−iq","x" "cos"]

test_list = []
for val in my_list :
  synsets = wn.synsets(val)
  value = len(synsets) > 0
  test_list.append(value)

# counting non-real words
false_count = test_list.count(False)/len(test_list)

if false_count > 0.5 :
    print("Sentence is not real")
else :
  print("Sentence is real")

Sentence is not real


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [214]:
def is_real_sent(doc):
  test_list = []
  for token in doc :
    synsets = wn.synsets(token.text)
    value = len(synsets) > 0
    test_list.append(value)

  # counting non-real words
  false_count = test_list.count(False)/len(test_list)

  if false_count >= 0.5 :
    return False
  else :
    return True

## Compiling Dataframe Cleaning Pipeline :

In [215]:
df

,sentences,entities,labels,tags
0,We eventually take the limit ε z/ε and t x/ε,"[ε, t]","[MATH_VAR, MATH_VAR]","[PRP, RB, VBP, DT, NN, NN, NNP, CC, NNP, NNP]"
1,"z → → → → ≡ ∼ Since Uz(x, z 0) ≈ ≈ 0","[z, ≡, z]","[MATH_VAR, MATH_OPP, MATH_VAR]","[NN, SYM, ``, ``, ``, FW, NNS, IN, JJ, NN, NNS..."
2,"we deﬁne the ﬂow at the surface by U(x, z = 0...","[deﬁne, z, =, ≡, U(x), +]","[MATH_VRB, MATH_VAR, MATH_OPP, MATH_OPP, MATH_...","[PRP, VBP, DT, NN, IN, DT, NN, IN, NN, NN, NN,..."
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O]","[MATH_VAR, MATH_VRB, MATH_VAR]","[NNP, VBZ, NN, VBZ, VBG, IN, NN, NNS, IN, UH, ..."
5,while δU(x/ε) varies over lengths of O(ε) comp...,[O(ε)],[MATH_VAR],"[IN, NN, VBZ, IN, NNS, IN, NN, JJ, IN, DT, JJ,..."
6,The amplitude of the slowly varying ﬂow U(x) i...,[U(x)],[MATH_VAR],"[DT, NN, IN, DT, RB, VBG, PRP$, NNP, VBZ, NNP]"
11,the boundary conditions (2.4) evaluated at z,[z],[MATH_VAR],"[DT, JJ, NNS, -LRB-, VBD, IN, NN]"
12,"0 become ∂tη(x, t) + ∇ · U(x) + √εδU(x/ε) ...","[∂tη(x, t), +, ∇, U(x), +, η(x)]","[MATH_VAR, MATH_OPP, MATH_VAR, MATH_VAR, MATH_...","[NFP, VB, NNP, NNP, NN, NFP, NNP, CC, NNP, -LR..."
13,"lim z→0− ∂zϕ(x, 0) ∂tϕ(x, t) + U(x) xϕ(x, t...","[∂tϕ(x, t), +, U(x), xϕ(x, t), +, xϕ(x, t)]","[MATH_VAR, MATH_OPP, MATH_VAR, MATH_VAR, MATH_...","[NNP, NNP, NNP, VBZ, NNP, CC, NNP, NNP, NFP, N..."
14,"ε∆η(x, t) · ∇ · ∇ (2.6) Although drift that...","[η(x, t), ∇, ∇]","[MATH_VAR, MATH_VAR, MATH_VAR]","[IN, NFP, ADD, NFP, ADD, NFP, IN, NN, WDT, VBZ..."


In [216]:
tags_to_exclude = []
for sent , ent , labels , tags in zip(df["sentences"],df["entities"],df["labels"],df["tags"]) :
    doc = custom_NER_tokenization(sent)
    if is_real_sent(doc) == False :
      # dropping the row of the false sentence
      df = df[df['sentences'] != sent]
    # if tags less than 4 drop it
    if len(tags)<4 :
      tags_to_exclude.append(tags)

df = df[~df['tags'].isin(tags_to_exclude)]

In [217]:
df

,sentences,entities,labels,tags
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O]","[MATH_VAR, MATH_VRB, MATH_VAR]","[NNP, VBZ, NN, VBZ, VBG, IN, NN, NNS, IN, UH, ..."
5,while δU(x/ε) varies over lengths of O(ε) comp...,[O(ε)],[MATH_VAR],"[IN, NN, VBZ, IN, NNS, IN, NN, JJ, IN, DT, JJ,..."
11,the boundary conditions (2.4) evaluated at z,[z],[MATH_VAR],"[DT, JJ, NNS, -LRB-, VBD, IN, NN]"
37,The Fourier integrals for η exclude q,"[η, q]","[MATH_VAR, MATH_VAR]","[DT, NNP, NNS, IN, NNP, NN, NN]"
56,Uniform drift yields the familiar capillary-gr...,"[≡, j]","[MATH_OPP, MATH_VAR]","[JJ, NN, VBZ, DT, JJ, NN, NN, NN, NN, VBP, NN]"
60,we wish to derive transport equations for surf...,[U],[MATH_VAR],"[PRP, VBP, TO, VB, NN, NNS, IN, NN, NNS, NNP, ..."


In [218]:
df[df['sentences'] != "The Fourier integrals for η exclude q"]

,sentences,entities,labels,tags
4,U(x) denotes surface ﬂows varying on length sc...,"[U(x), denotes, O]","[MATH_VAR, MATH_VRB, MATH_VAR]","[NNP, VBZ, NN, VBZ, VBG, IN, NN, NNS, IN, UH, ..."
5,while δU(x/ε) varies over lengths of O(ε) comp...,[O(ε)],[MATH_VAR],"[IN, NN, VBZ, IN, NNS, IN, NN, JJ, IN, DT, JJ,..."
11,the boundary conditions (2.4) evaluated at z,[z],[MATH_VAR],"[DT, JJ, NNS, -LRB-, VBD, IN, NN]"
37,The Fourier integrals for η exclude q,"[η, q]","[MATH_VAR, MATH_VAR]","[DT, NNP, NNS, IN, NNP, NN, NN]"
56,Uniform drift yields the familiar capillary-gr...,"[≡, j]","[MATH_OPP, MATH_VAR]","[JJ, NN, VBZ, DT, JJ, NN, NN, NN, NN, VBP, NN]"
60,we wish to derive transport equations for surf...,[U],[MATH_VAR],"[PRP, VBP, TO, VB, NN, NNS, IN, NN, NNS, NNP, ..."


In [219]:
df['entities'].apply(lambda ent: len(ent) > 0)

4     True
5     True
11    True
37    True
56    True
60    True
Name: entities, dtype: bool

In [220]:
condition1 = df['entities'].apply(lambda ent: len(ent) > 0)
condition2 = df['tags'].apply(lambda tags: len(tags) > 4)
df = df[condition1 & condition2]

In [221]:
def clean_df(df) :

  # I - getting rid of empty entities and tags :
  condition = df['entities'].apply(lambda ent: len(ent) > 0)
  condition2 = df['tags'].apply(lambda tags: len(tags) > 4)
  df = df[condition1 & condition2]


  # II - Cleaning dataframe ffrom marginal POS tags
  target_words_A = {'A','a'}
  target_words_I = {'i' ,'I'}
  sent_to_exclude = []

  for sent , ent , labels , tags in zip(df["sentences"],df["entities"],df["labels"],df["tags"]) :
    doc = custom_NER_tokenization(sent)

    if is_real_sent(doc) == False :
      # dropping the row of the false sentence
      df = df[df['sentences'] != sent]

    else :
      token_list = [token for token in doc]
      # looping over tokens
      for token in doc :
        token_index = token_list.index(token)
        # getting rid of marginal "a" and its POS tag
        if (token.text in target_words_A) & (token.text in ent) & (tags[token_index] == "DT") :
          ent_index = ent.index(token.text)
          ent.pop(ent_index)
          labels.pop(ent_index)

        # getting rid of marginal "i" and its POS tag
        if (token.text in target_words_I) & (token.text in ent) & (token_index == 0) :
          ent_index = ent.index(token.text)
          ent.pop(ent_index)
          labels.pop(ent_index)

  return df

# Getting `var-name` compound :

### Test Case I :

In [222]:
sent1_1 = "The velocity of an object v"
sent1_2 = "Velocity v of an object"

sent2_1 = "The variables x and y are respectfully the mass and velocity"
sent2_2 = "where a is velocity and b is mass"


tag1_1 = POS_tag(sent1_1)
tag1_2 = POS_tag(sent1_2)


tag2_1 = POS_tag(sent2_1)
tag2_2 = POS_tag(sent2_2)


tag_test_list = [(tag1_1,tag1_2),(tag2_1,tag2_2)]


In [223]:
for (tag1,tag2) in tag_test_list :
  print(f"\ntag case I :\n")
  print(tag1["words"])
  print(tag1["tag"])
  print("*"*20)
  print(f"\ntag case II :\n")
  print(tag2["words"])
  print(tag2["tag"])
  print("-"*100)


tag case I :

['The', 'velocity', 'of', 'an', 'object', 'v']
['DT', 'NN', 'IN', 'DT', 'NN', 'IN']
********************

tag case II :

['Velocity', 'v', 'of', 'an', 'object']
['NN', 'NN', 'IN', 'DT', 'NN']
----------------------------------------------------------------------------------------------------

tag case I :

['The', 'variables', 'x', 'and', 'y', 'are', 'respectfully', 'the', 'mass', 'and', 'velocity']
['DT', 'NNS', 'NNP', 'CC', 'NNP', 'VBP', 'RB', 'DT', 'NN', 'CC', 'NN']
********************

tag case II :

['where', 'a', 'is', 'velocity', 'and', 'b', 'is', 'mass']
['WRB', 'DT', 'VBZ', 'NN', 'CC', 'NN', 'VBZ', 'JJ']
----------------------------------------------------------------------------------------------------


In [224]:
sent_list = [sent1_1,sent1_2,sent2_1,sent2_2]
docs = [custom_NER_tokenization(sent,True) for sent in sent_list]
docs

[The velocity of an object v ,
 Velocity v of an object ,
 The variables x and y are respectfully the mass and velocity ,
 where a is velocity and b is mass ]

In [225]:
for doc in docs :
  for ent in doc.ents :
    print(ent.text,ent.label_)

v MATH_VAR
v MATH_VAR
x MATH_VAR
y MATH_VAR
a MATH_VAR
b MATH_VAR


### Test Case II :

In [275]:
sentI = "X corresponds to the name"
sentII = "The parameter X is taken to be something special"
sentIII = "The name of the variable is denoted as something"

tagI = POS_tag(sentI)
tagII= POS_tag(sentII)
tagIII = POS_tag(sentIII)

tagZ_test_list = [tagI,tagII,tagIII]

for tag in tagZ_test_list :
  print(tag["words"])
  print(tag["tag"])
  print("*"*20)

['X', 'corresponds', 'to', 'the', 'name']
['NN', 'VBZ', 'IN', 'DT', 'NN']
********************
['The', 'parameter', 'X', 'is', 'taken', 'to', 'be', 'something', 'special']
['DT', 'NN', 'NNP', 'VBZ', 'VBN', 'TO', 'VB', 'NN', 'JJ']
********************
['The', 'name', 'of', 'the', 'variable', 'is', 'denoted', 'as', 'something']
['DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'VBN', 'IN', 'NN']
********************


In [280]:
tokens = ["X","corresponds","to","the","name"]
print(("corresponds".split(" "))[0])

corresponds


In [281]:
var_dict = {"sentence": [], "vars": [], "names": []}
for sent, ent, labels, tags in zip(df["sentences"], df["entities"], df["labels"], df["tags"]):
    doc = custom_NER_tokenization(sent)
    word_list = [token.text for token in doc]
    variables = []
    names = []

    if 'MATH_VRB' in labels and 'and' not in word_list:
      # Getting index of label
      ent_idx = labels.index('MATH_VRB')
      # getting entitie of the label
      ent = ent[ent_idx]
      my_ent = ent.split(" ")[0]
      # getting index of the variable in word list
      vrb_word_idx = word_list.index(my_ent)
      if "MATH_VAR" in labels :
        var_idx = labels.index('MATH_VRB')
        # getting entity of the var
        var_ent = ent[var_idx]

        # variable after verb
        if var_idx > ent_idx :
          words = [word.lower() for word in word_list]

          if "the" in words:
            # Get the first occurrence of the word "the"
            the_idx = words.index("the")
            name = " ".join(word_list[the_idx:vrb_word_idx])

            variables.append(var_ent)
            names.append(name)
          else:
            NN_idx = tags.index("NN")
            name = " ".join(word_list[NN_idx:vrb_word_idx])
            variables.append(var_ent)
            names.append(name)
        # variable before verb
        elif var_idx < ent_idx :

          words = [word.lower() for word in word_list]
          word_edited = words[vrb_word_idx:]

          if "the" in word_edited:
            # Get the first occurrence of the word "the"
            the_idx = word_edited.index("the")
            name = " ".join(word_edited[the_idx:])

            variables.append(var_ent)
            names.append(name)
          else:
            NN_idx = word_edited.index("NN")
            name = " ".join(word_edited[NN_idx:vrb_word_idx])
            variables.append(var_ent)
            names.append(name)

    add_to_dict(var_dict,sent,variables,names)

In [282]:
pd.DataFrame(var_dict)

,sentence,vars,names
0,U(x) denotes surface ﬂows varying on length sc...,[],[]
1,while δU(x/ε) varies over lengths of O(ε) comp...,[],[]
2,the boundary conditions (2.4) evaluated at z,[],[]
3,The Fourier integrals for η exclude q,[],[]
4,Uniform drift yields the familiar capillary-gr...,[],[]
5,we wish to derive transport equations for surf...,[],[]


### Test Case II :

In [226]:
test = ["hello","and","tai","bee","and","beb"]
last_index = len(test) - 1 - list(reversed(test)).index("and")
last_index

4

In [227]:
# doing tests when the variable is in the (begining,middle,end)
sent1 = "v is the velocity of an object"
sent2 = "The variable v is the velocity of an object"
sent3 = "The velocity of an object v"
sent4 = "Velocity of an object v"
sent5 = "Variables a and b are for mass bassis and velocity"
sent6 = "Variables a is mass and b is velocity"


sent_list2 = [sent1,sent2,sent3,sent4,sent5,sent6]
tags2 = [POS_tag(sent) for sent in sent_list2]

In [228]:
for tag in tags2 :
  print(tag["words"])
  print(tag["tag"])
  print("-"*50)

['v', 'is', 'the', 'velocity', 'of', 'an', 'object']
['NN', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN']
--------------------------------------------------
['The', 'variable', 'v', 'is', 'the', 'velocity', 'of', 'an', 'object']
['DT', 'JJ', 'NN', 'VBZ', 'DT', 'NN', 'IN', 'DT', 'NN']
--------------------------------------------------
['The', 'velocity', 'of', 'an', 'object', 'v']
['DT', 'NN', 'IN', 'DT', 'NN', 'IN']
--------------------------------------------------
['Velocity', 'of', 'an', 'object', 'v']
['NN', 'IN', 'DT', 'NN', 'IN']
--------------------------------------------------
['Variables', 'a', 'and', 'b', 'are', 'for', 'mass', 'bassis', 'and', 'velocity']
['NNS', 'DT', 'CC', 'NN', 'VBP', 'IN', 'JJ', 'NN', 'CC', 'NN']
--------------------------------------------------
['Variables', 'a', 'is', 'mass', 'and', 'b', 'is', 'velocity']
['NNS', 'DT', 'VBZ', 'NN', 'CC', 'NN', 'VBZ', 'NN']
--------------------------------------------------


In [229]:
docs2 = [custom_NER_tokenization(sent,True) for sent in sent_list2]
docs2

[v is the velocity of an object ,
 The variable v is the velocity of an object ,
 The velocity of an object v ,
 Velocity of an object v ,
 Variables a and b are for mass bassis and velocity ,
 Variables a is mass and b is velocity ]

In [230]:
my_list = []
var_list = []
ent_list = []
for doc in docs2 :
  sub_var_list = []
  sub_ent_list = []
  for ent in doc.ents :
    sub_var_list.append(ent.text)
    sub_ent_list.append(ent.label_)
  var_list.append(sub_var_list)
  ent_list.append(sub_ent_list)


print(var_list)
print(ent_list)

[['v'], ['v'], ['v'], ['v'], ['a', 'b'], ['a', 'b']]
[['MATH_VAR'], ['MATH_VAR'], ['MATH_VAR'], ['MATH_VAR'], ['MATH_VAR', 'MATH_VAR'], ['MATH_VAR', 'MATH_VAR']]


In [234]:
variables

[('v', 'the velocity of an object'),
 ('v', 'the velocity of an object'),
 ('v', 'The velocity of an object'),
 ('v', 'Velocity of an object'),
 ('a', 'for mass bassis'),
 ('b', 'velocity'),
 ('a', 'mass'),
 ('b', 'velocity')]

## Applying variables extraction test cases :

In [235]:
def add_to_dict(dictionary,sent,vars,names) :
  dictionary["vars"].append(vars)
  dictionary["names"].append(names)
  dictionary["sentence"].append(sent)

In [236]:
def var_begin (word_list,tags,var_idx,var,variables,names) :
  # Checking there is a verb after the variable (usually an introductory verb)
  condition1 = tags[var_idx + 1] == "VBZ"
  # Checking if there is a noun after that verb
  condition1_1 = tags[var_idx + 2] == "NN"
  # Or checking if there is "the" after that verb
  condition1_2 = tags[var_idx + 2] == "the"

  if condition1 and (condition1_1 or condition1_2):
    name = " ".join(word_list[var_idx + 2:])
    variables.append(var)
    names.append(name)

  return variables,names

In [237]:
def var_end (word_list,tags,var_idx,var,variables,names) :
  # Checking if there is a verb before the variable (usually an introductory verb)
  condition2_1 = tags[var_idx - 1] == "VBZ"
  # Checking if there is a noun before the variable
  condition2_2 = tags[var_idx - 1] == "NN"
  if condition2_1 or condition2_2:
    # Lowercasing words
    words = [word.lower() for word in word_list]
    if "the" in words:
      # Get the first occurrence of the word "the"
      the_idx = words.index("the")
      name = " ".join(word_list[the_idx:var_idx])
      variables.append(var)
      names.append(name)
    else:
      NN_idx = tags.index("NN")
      name = " ".join(word_list[NN_idx:var_idx])
      variables.append(var)
      names.append(name)

  return variables,names

In [238]:
def var_middle(word_list,tags,var_idx,var,variables,names) :
  if var_idx >= len(word_list) - 2 :
    condition3_1 = tags[var_idx + 1] == "VBZ"
    condition3_2 = word_list[var_idx + 2] == "the"
    if condition3_1 and condition3_2:
      name = " ".join(word_list[var_idx + 2:])
      variables.append(var)
      names.append(name)

    else:
      words = [word.lower() for word in word_list]

      if "the" in words:
        the_idx = words.index("the")
        name = " ".join(word_list[the_idx:var_idx])
        variables.append(var)
        names.append(name)

      else:
        NN_idx = tags.index("NN")
        name = " ".join(word_list[NN_idx:var_idx])
        variables.append(var)
        names.append(name)

  return variables,names

In [239]:
def single_var(sent,labels,ent,tags,variables,names,word_list,dictionnary,dict_func) :
  # Getting index of label
  ent_idx = labels.index('MATH_VAR')
  # Getting variable from entities
  var = ent[ent_idx]
  # Getting variable position in sentence
  var_idx = word_list.index(var)
  # CASE I:
  if var_idx == 0:
    variables,names = var_begin(word_list,tags,var_idx,var,variables,names)
    dict_func(dictionnary,sent,variables,names)
  # CASE II :
  elif var_idx == len(word_list) - 1 or var_idx == len(word_list) - 2 :
    variables,names = var_end(word_list,tags,var_idx,var,variables,names)
    dict_func(dictionnary,sent,variables,names)
  else:
    variables,names = var_middle(word_list,tags,var_idx,var,variables,names)
    dict_func(dictionnary,sent,variables,names)

In [240]:
def multiple_vars(word_list,tags,var_idx,variables,names,dictionnary,dict_func) :
  if word_list.count('and') == 2 & labels.count("MATH_VAR") ==2:
    # Getting the second "and" index
    last_index = len(word_list) - 1 - list(reversed(word_list)).index("and")
    last_verb = len(tags) - 1 - list(reversed(tags)).index("VBP")
    tag_index = tags[last_index]
    name_1 = " ".join(word_list[last_verb + 1:last_index])
    name_2 = " ".join(word_list[last_index + 1:])
    names.append([name_1, name_2])
    math_ents = []
    for index, label in enumerate(labels):
      if label == "math_var":
        math_ents.append(ent[index])
    variables.append([math_ents[0], math_ents[1]])

  elif word_list.count('and') == 1 & labels.count("MATH_VAR") == 2:
    and_ind = word_list.index("and")
    math_ents = []
    for index, label in enumerate(labels):
      if label == "math_var":
        math_ents.append(entities[index])

    var_1_ind = word_list.index(math_ents[0])
    var_2_ind = word_list.index(math_ents[1])
    name_1 = " ".join(word_list[var_1_ind + 2:and_ind])
    name_2 = " ".join(word_list[var_2_ind + 2:])
    names.append([name_1, name_2])
    variables.append([math_ents[0], math_ents[1]])

  dict_func(dictionnary,sent,variables,names)

In [241]:
def get_vars_names(dictionnary,dataframe) :
  for sent, ent, labels, tags in zip(dataframe["sentences"], dataframe["entities"], dataframe["labels"], dataframe["tags"]):
    doc = custom_NER_tokenization(sent)
    word_list = [token.text for token in doc]
    variables = []
    names = []

    if 'MATH_VAR' in labels and 'and' not in word_list:
      single_var(sent,labels,ent,tags,variables,names,word_list,dictionnary,add_to_dict)

    elif 'MATH_VAR' in labels and 'and' in word_list :
      multiple_vars(word_list,tags,var_idx,variables,names,dictionnary,add_to_dict)

    elif 'MATH_VRB' in labels  and 'and' not in word_list:


  return pd.DataFrame(dictionnary)

In [242]:
#dicto = {"sentence": [], "vars": [], "names": []}
#new_df = get_vars_names(dicto,df)

In [243]:
#new_df

In [244]:
"""dicto = {"sentence": [], "vars": [], "names": []}

for sent, ent, labels, tags in zip(df["sentences"], df["entities"], df["labels"], df["tags"]):
    doc = custom_NER_tokenization(sent)
    word_list = [token.text for token in doc]
    variables = []
    names = []

    if 'MATH_VAR' in labels and 'and' not in word_list:
        # Getting index of label
        ent_idx = labels.index('MATH_VAR')
        # Getting variable from entities
        var = ent[ent_idx]
        # Getting variable position in sentence
        var_idx = word_list.index(var)

        # CASE I:
        if var_idx == 0:
            # Checking there is a verb after the variable (usually an introductory verb)
            condition1 = tags[var_idx + 1] == "VBZ"
            # Checking if there is a noun after that verb
            condition1_1 = tags[var_idx + 2] == "NN"
            # Or checking if there is "the" after that verb
            condition1_2 = tags[var_idx + 2] == "the"

            if condition1 and (condition1_1 or condition1_2):
                name = " ".join(word_list[var_idx + 2:])
                variables.append(var)
                names.append(name)

            add_to_dict(dicto,sent,variables,names)

        elif var_idx == len(word_list) - 1:
            # Checking if there is a verb before the variable (usually an introductory verb)
            condition2_1 = tags[var_idx - 1] == "VBZ"
            # Checking if there is a noun before the variable
            condition2_2 = tags[var_idx - 1] == "NN"

            if condition2_1 or condition2_2:
                # Lowercasing words
                words = [word.lower() for word in word_list]

                if "the" in words:
                    # Get the first occurrence of the word "the"
                    the_idx = words.index("the")
                    name = " ".join(word_list[the_idx:var_idx])
                    variables.append(var)
                    names.append(name)
                else:
                    NN_idx = tags.index("NN")
                    name = " ".join(word_list[NN_idx:var_idx])
                    variables.append(var)
                    names.append(name)

                add_to_dict(dicto,sent,variables,names)

        else:
            condition3_1 = tags[var_idx + 1] == "VBZ"
            condition3_2 = word_list[var_idx + 2] == "the"

            if condition3_1 and condition3_2:
                name = " ".join(word_list[var_idx + 2:])
                variables.append((var, name))
            else:
                words = [word.lower() for word in word_list]

                if "the" in words:
                    the_idx = words.index("the")
                    name = " ".join(word_list[the_idx:var_idx])
                    variables.append(var)
                    names.append(name)
                else:
                    NN_idx = tags.index("NN")
                    name = " ".join(word_list[NN_idx:var_idx])
                    variables.append(var)
                    names.append(name)

                add_to_dict(dicto,sent,variables,names)

    else:
        if word_list.count('and') == 2:
            # Getting the second "and" index
            last_index = len(word_list) - 1 - list(reversed(word_list)).index("and")
            last_verb = len(tags) - 1 - list(reversed(tags)).index("VBP")

            tag_index = tags[last_index]
            name_1 = " ".join(word_list[last_verb + 1:last_index])
            name_2 = " ".join(word_list[last_index + 1:])
            names.append([name_1, name_2])
            variables.append([ent[0], ent[1]])

        elif word_list.count('and') == 1:
            and_ind = word_list.index("and")
            var_1_ind = word_list.index(ent)
            var_2_ind = word_list.index(ent)
            name_1 = " ".join(word_list[var_1_ind + 2:and_ind])
            name_2 = " ".join(word_list[var_2_ind + 2:])
            names.append([name_1, name_2])
            variables.append([ent[0], ent[1]])

        add_to_dict(dicto,sent,variables,names)"""

'dicto = {"sentence": [], "vars": [], "names": []}\n\nfor sent, ent, labels, tags in zip(df["sentences"], df["entities"], df["labels"], df["tags"]):\n    doc = custom_NER_tokenization(sent)\n    word_list = [token.text for token in doc]\n    variables = []\n    names = []\n\n    if \'MATH_VAR\' in labels and \'and\' not in word_list:\n        # Getting index of label\n        ent_idx = labels.index(\'MATH_VAR\')\n        # Getting variable from entities\n        var = ent[ent_idx]\n        # Getting variable position in sentence\n        var_idx = word_list.index(var)\n\n        # CASE I:\n        if var_idx == 0:\n            # Checking there is a verb after the variable (usually an introductory verb)\n            condition1 = tags[var_idx + 1] == "VBZ"\n            # Checking if there is a noun after that verb\n            condition1_1 = tags[var_idx + 2] == "NN"\n            # Or checking if there is "the" after that verb\n            condition1_2 = tags[var_idx + 2] == "the"\n\n   

In [245]:
#pd.DataFrame(dicto)

## Creating Feature Extraction Pipeline :

In [246]:
my_text = read("/content/drive/MyDrive/UH - Final Year Project/Dataset/cleaned_text/0001001v1.txt")
print(my_text[3000:4000])

ei ).
Nonetheless, water wave propagation over random               Bal & Chou  static underlying currents that vary on both large and small length scales, and their interactions, have received relatively less attention.
In this paper, we will only consider static irrotational currents, but derive the transport equations for surface waves in the presence of underlying ﬂows that vary on both long and short (on the order of the wavelength) length scales.
Rather than computing wave scattering from speciﬁc static ﬂow conﬁgurations (Gerber , Trulson & Mei , Fabrikant & Raevsky ), we take a statistical approach by considering ensemble averages over realisations of the static randomness.
Diﬀerent statistical approaches have been applied to wave propagation over a random depth (Elter & Molyneux ), third sound localization in superﬂuid Helium ﬁlms (Kleinert ), and wave diﬀusion in the presence of turbulent ﬂows (Howe , Rayevskiy , Fannjiang & Ryzhik ).
In the next section we derive the linearis

In [247]:
labels = ["MATH_VAR","MATH_STRUCT", "MATH_SYMBOL" , "MATH_OPP" ,"MATH_VRB"]
pattern = r'[,=]\s*(?![^()]*\))'
my_dict = {"sentence": [], "vars": [], "names": []}

In [248]:
# defining sample text snippet
sample_text = """We eventually take the limit ε  z/ε and t  x/ε, z  →  →  →  →   ≡  ∼  Since Uz(x, z  0)  ≈  ≈  0, we deﬁne the ﬂow at the surface by  U(x, z = 0)  ≡  U(x) + √εδU(x/ε).
(2.5)  In these rescaled coordinates, U(x) denotes surface ﬂows varying on length scales of O much greater than a typical wavelength, while δU(x/ε) varies over lengths of O(ε) comparable to a typical wavelength.
The amplitude of the slowly varying ﬂow U(x) is O(ε0), while that of the rapidly varying ﬂow δU(x/ε), is assumed to be of O(√ε).
A more detailed discussion of the physical motivation for considering the √ε scaling is deferred to the Results and Discussion.
After rescaling, the boundary conditions (2.4) evaluated at z = 0 become  ∂tη(x, t) +  ∇  ·  U(x) + √εδU(x/ε) (cid:1)  (cid:2)(cid:0)  η(x) (cid:3)  = lim  z→0− ∂zϕ(x, 0)  ∂tϕ(x, t) + U(x)  xϕ(x, t) + √εδU(x/ε)  xϕ(x, t) = ε∆η(x, t)  · ∇  · ∇  (2.6) Although drift that varies slowly along one wavelength can be treated with characteristics and WKB theory, random ﬂows varying on the wavelength scale require a statistical approach.
Without loss of generality, we choose δU to have zero mean and an isotropic x′ two-point correlation function ), where (i, j) = (1, 2) and | denotes an ensemble average over realisations of δU(x).
.
.
.
δUi(x)δUj (x′)
We now deﬁne the spatial Fourier decompositions for the dynamical wave variables  Rij( |  i ≡  −  ε−1η(x, t).
−  ϕ(x,  −  h 6 z 6 ζ, t) =  ϕ(q, t)e−iq·x cosh q(h + z)  cosh qh  Zq  ,  η(x, t) =  Zq  η(q, t)e−iq·x,  the static surface ﬂows  U(x) =  Zq  U(q)e−iq·x,  δU  ε (cid:17)  (cid:16)  =  Zq  δU(q)e−iq·x/ε,  and the correlations  Rij (x) =  Zq  Rij (q)e−iq·x,  (2.7)  (2.8)  (2.9)  where q = (q1, q2) is an in-plane two dimensional wavevector, q  2, and (2π)−2 dq1dq2.
The Fourier integrals for η exclude q = 0 due to the incompress- q ≡ x η(x, t) = 0, while the q = 0 mode for ϕ gives an irrelevant constant ibility constraint shift to the velocity potential.
Note that ϕ in (2.7) manifestly satisﬁes (2.1).
Substituting  |  ≡ |  1 + q2 q2   =
(2.8) into the boundary conditions (2.4), we obtain,  Water wave transport  ∂tη(k, t)   −  Zq  η(k  −  q)U(q)   ·  −  i√ε  Zq  η(k  −  q/ε)δU(q)  ·  k = ϕ(k, t) k tanh εkh  ∂tϕ(k, t)   −  Zq  U(q)  (k  ·  −  q) ϕ(k  q)  −  −  i√ε  Zq  δU(q)  (k  ·  −  q/ε) ϕ(k  q/ε)  −  where the δU(q) are correlated according to  δUi(p)δUj(q)  )δ(p + q).
= Rij ( | |  (εk2 + ε−1)η(k).
=  −  (2.10)  (2.11)  Since the correlation Rij (x) is symmetric in i , Rij( | | | In the case where δU = 0 and U(x)  ) is real.
|  −  U0 is strictly uniform, equations (2.10) can be simpliﬁed by assuming a e−iωt dependence for all dynamical variables.
Uniform drift yields the familiar capillary-gravity wave dispersion relation  ≡  j, and depends only upon the magnitude  ↔  ω(k) =  (k3 + k) tanh kh + U0 ·    Ω(k) + U0 ·  k.  ≡  (2.12)  However, for what follows, we wish to derive transport equations for surface waves (action, energy, intensity) in the presence of a spatially varying drift containing two length scales: U = U(x) + √εδU(x/ε).
"""

In [249]:
from datetime import datetime

In [250]:
"""# pipeline

# start timer
start_time = datetime.now()
df1_test = create_df(sample_text,pattern,labels)
add_tags(df1_test)
df2_test = clean_df(df1_test)
df3 = get_vars_names(my_dict,df2_test)
# end timer
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time test: {elapsed_time}")"""

'# pipeline\n\n# start timer\nstart_time = datetime.now()\ndf1_test = create_df(sample_text,pattern,labels)\nadd_tags(df1_test)\ndf2_test = clean_df(df1_test)\ndf3 = get_vars_names(my_dict,df2_test)\n# end timer\nend_time = datetime.now()\nelapsed_time = end_time - start_time\nprint(f"Elapsed time test: {elapsed_time}")'

In [251]:
#df2_test

In [252]:
"""labels = ["MATH_VAR","MATH_STRUCT", "MATH_SYMBOL" , "MATH_OPP" ,"MATH_VRB"]
pattern = r'[,=]\s*(?![^()]*\))'
my_dict = {"sentence": [], "vars": [], "names": []}"""

'labels = ["MATH_VAR","MATH_STRUCT", "MATH_SYMBOL" , "MATH_OPP" ,"MATH_VRB"]\npattern = r\'[,=]\\s*(?![^()]*\\))\'\nmy_dict = {"sentence": [], "vars": [], "names": []}'

In [253]:
"""# pipeline

# start timer
start_time = datetime.now()
df1 = create_df(my_text,pattern,labels)
add_tags(df1)
df2 = clean_df(df1)
df3 = get_vars_names(my_dict,df2)
# end timer
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")"""

'# pipeline\n\n# start timer\nstart_time = datetime.now()\ndf1 = create_df(my_text,pattern,labels)\nadd_tags(df1)\ndf2 = clean_df(df1)\ndf3 = get_vars_names(my_dict,df2)\n# end timer\nend_time = datetime.now()\nelapsed_time = end_time - start_time\nprint(f"Elapsed time: {elapsed_time}")'

In [254]:
#df3.tail(40)

In [255]:
#df3.to_csv("dataframe.csv", encoding='utf-8')

In [256]:
#df2

In [257]:
#df3

In [258]:
my_text2 = read("/content/drive/MyDrive/UH - Final Year Project/Dataset/cleaned_text/0001024v1.txt")
print(my_text2[3000:4000])

n the nonsymmetric form.
The  symmetric form was able to reproduce accurate variational results very precisely for singlet  Ps-H binding and resonance energies; the nonsymmetric form failed to yield such results.
In view of this we reinvestigate the problem of low-energy Ps scattering by He  using the  symmetric exchange potential employing the three-Ps-state model used before.
We study how  the low-energy cross sections for Ps-H and Ps-He change with the variation of the parameter  C of the potential.
Then we also apply this exchange potential to the study of low-energy Ps  scattering by Ne and Ar using a static-exchange model.
The present calculation accounts for  the measured low-energy cross sections [1, 2, 3] satisfactorily for He, Ne, and Ar.
The total wave function of the Ps-target system is expanded in terms of the Ps eigenstates  as  Ψ±(r1, ..., rN; rN+1, x) =  Fν(ρN+1)χν(tN+1)φ0(r1, ..., rj, ..., rN)  ν (cid:20)  +  (−1)SN+1,jFν(ρj)χν(tj)φ0(r1, ..., rN+1, ..., rN)  Xj=1  , (

In [259]:
labels = ["MATH_VAR","MATH_STRUCT", "MATH_SYMBOL" , "MATH_OPP" ,"MATH_VRB"]
pattern = r'[,=]\s*(?![^()]*\))'
dicta = {"sentence": [], "vars": [], "names": []}

In [260]:
# pipeline

# start timer
start_time = datetime.now()
df11 = create_df(my_text2,pattern,labels)
add_tags(df11)
df22 = clean_df(df11)
df33 = get_vars_names(dicta,df22)
# end timer
end_time = datetime.now()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time}")

Elapsed time: 0:10:54.001466


In [261]:
df33.tail(40)

,sentence,vars,names
0,Scattering of positronium by H,[],[]
1,U(x) + √εδU(x/ε).,[],[]
2,there have been precise measurements of low-e...,[],[]
3,U(x) + √εδU(x/ε).,[],[]
4,the number of channels grow as N 2.,[],[]
5,U(x) + √εδU(x/ε).,[],[]
6,The nonsymmetric potential has been applied fo...,[],[]
7,U(x) + √εδU(x/ε).,[],[]
8,In a subsequent application of this model pote...,[],[]
9,N denote the target electron coordinates,[],[]


In [269]:
df33.iloc[11].names

['the resulting momentum-space Lippmann-Schwinger scattering equation for a particular total electronic spin state']

In [262]:
df33.tail(30)

,sentence,vars,names
0,Scattering of positronium by H,[],[]
1,U(x) + √εδU(x/ε).,[],[]
2,there have been precise measurements of low-e...,[],[]
3,U(x) + √εδU(x/ε).,[],[]
4,the number of channels grow as N 2.,[],[]
5,U(x) + √εδU(x/ε).,[],[]
6,The nonsymmetric potential has been applied fo...,[],[]
7,U(x) + √εδU(x/ε).,[],[]
8,In a subsequent application of this model pote...,[],[]
9,N denote the target electron coordinates,[],[]


In [271]:
for sent in df33["sentence"] :
  print(sent)

Scattering of positronium by H
U(x) + √εδU(x/ε).
there have been precise measurements  of low-energy ortho-Ps scattering by H2
U(x) + √εδU(x/ε).
the number of channels grow as N 2.
U(x) + √εδU(x/ε).
The nonsymmetric potential has been applied for the  study of Ps scattering by H 
U(x) + √εδU(x/ε).
In a subsequent application of this model potential to Ps scattering by H 
N denote the target electron coordinates
U(x) + √εδU(x/ε).
the resulting momentum-space Lippmann-Schwinger scattering equation for a  particular total electronic spin state S can
U(x) + √εδU(x/ε).
U(x) + √εδU(x/ε).
there will be only one scattering equation  corresponding to total electronic spin S 
in the sum over j in Eq.
For Ps scattering from a H orbital
the parameter C is taken to be unity  [4
A variation of C from unity leads to a variation of the binding energy parameters (α2, β2 etc.)
We have turned this ﬂexibility to good advantage by obtaining precise agreement  with low-energy results of Ps scattering by H
U

# Other :

In [263]:
"""# testing how lemma's of these verbs would be
verb = ["called", "defined as", "used for", "termed as", "represents", "denotes", "stands for", "refers to",
              "corresponds to", "signifies", "designates", "characterizes as","identified as", "labelled as", "named",
              "outlined as", "identified with", "associated with", "portrays", "symbolizes", "denoted", "denotes","given by"]


math_vrbs = " ".join(verb)
doc = nlp(math_vrbs)
for token in doc :
  print(token,"===>",token.lemma_)"""

'# testing how lemma\'s of these verbs would be\nverb = ["called", "defined as", "used for", "termed as", "represents", "denotes", "stands for", "refers to",\n              "corresponds to", "signifies", "designates", "characterizes as","identified as", "labelled as", "named",\n              "outlined as", "identified with", "associated with", "portrays", "symbolizes", "denoted", "denotes","given by"]\n\n\nmath_vrbs = " ".join(verb)\ndoc = nlp(math_vrbs)\nfor token in doc :\n  print(token,"===>",token.lemma_)'

In [264]:
"""import spacy

nlp = spacy.load("en_core_web_sm")

is_a_stop_word = nlp.vocab["a"].is_stop
is_i_stop_word = nlp.vocab["i"].is_stop

print(f"'a' is a stop word : {is_a_stop_word}")
print(f"'i' a stop word : {is_i_stop_word}")"""

'import spacy\n\nnlp = spacy.load("en_core_web_sm")\n\nis_a_stop_word = nlp.vocab["a"].is_stop\nis_i_stop_word = nlp.vocab["i"].is_stop\n\nprint(f"\'a\' is a stop word : {is_a_stop_word}")\nprint(f"\'i\' a stop word : {is_i_stop_word}")'